The second homework

In [1]:
import wget

data = 'https://raw.githubusercontent.com/alexeygrigorev/datasets/master/housing.csv'
!wget $data 

--2023-09-25 16:05:12--  https://raw.githubusercontent.com/alexeygrigorev/datasets/master/housing.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1423529 (1.4M) [text/plain]
Saving to: 'housing.csv.3'

     0K .......... .......... .......... .......... ..........  3%  743K 2s
    50K .......... .......... .......... .......... ..........  7% 2.74M 1s
   100K .......... .......... .......... .......... .......... 10% 1.24M 1s
   150K .......... .......... .......... .......... .......... 14% 2.78M 1s
   200K .......... .......... .......... .......... .......... 17% 1.62M 1s
   250K .......... .......... .......... .......... .......... 21% 3.16M 1s
   300K .......... .......... .......... .......... .......... 25% 3.38M 1s
   350K .......... .......... 

In [ ]:
import pandas as pd
import numpy as np



df = pd.read_csv('housing.csv')
df.head()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

sns.histplot(df.median_house_value, bins=50)

In [ ]:
df_short = df[df.ocean_proximity.isin(['<1H OCEAN','INLAND'])]
df_short.tail()

#df_short = df_short[['latitude','longitude','housing_median_age','total_rooms','total_bedrooms','population','households','median_income','median_house_value']]
#df_short.head()

del df_short['ocean_proximity']
df_short.head()

df_short.isnull().sum()

In [ ]:
df_short['population'].median()

In [ ]:
n = df_short.shape[0]
n_train = int(n * 0.6)
n_val = int(n * 0.2)

idx = np.arange(n)
np.random.seed(42)
np.random.shuffle(idx)

df_train = df_short.iloc[idx[:n_train]]
df_val = df_short.iloc[idx[n_train:n_train+n_val]]
df_test = df_short.iloc[idx[n_train+n_val:]]

print(n)
print(len(df_train)+len(df_val)+len(df_test))
df_train.head()

In [ ]:
df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)
df_train.head()

y_train = np.log1p(df_train.median_house_value.values)
y_val = np.log1p(df_val.median_house_value.values)
y_test = np.log1p(df_test.median_house_value.values)

del df_train['median_house_value']
del df_val['median_house_value']
del df_test['median_house_value']
df_train.head()


In [ ]:
len(y_train)

In [ ]:
def train_linear_regression(X, y):
    ones = np.ones(X.shape[0])
    X = np.column_stack([ones, X])

    XTX = X.T.dot(X)
    XTX_inv = np.linalg.inv(XTX)
    w_full = XTX_inv.dot(X.T).dot(y)
    
    return w_full[0], w_full[1:]

df_X_train = df_train.fillna(0)
df_X_train.isna().sum()

X_train = df_X_train.values
w0, w = train_linear_regression(X_train, y_train)
print(w0,w)


In [ ]:
def rmse(y, y_pred):
    se = (y - y_pred) ** 2
    mse = se.mean()
    return np.sqrt(mse)

rmse(y_train, y_pred)

In [ ]:
mean_tb = int(df_train['total_bedrooms'].mean())
mean_tb


In [ ]:
X_train_mean = df_train.fillna(mean_tb).values

w0_mean, w_mean = train_linear_regression(X_train_mean, y_train)
w0_mean, w_mean

In [ ]:
df_train.head()


In [ ]:
df_val.head()

In [ ]:
X_val = df_val.fillna(0).values
y_pred = w0 + X_val.dot(w)
rmse(y_val, y_pred)

In [ ]:
X_val_mean = df_val.fillna(mean_tb).values
y_pred_mean = w0_mean + X_val_mean.dot(w_mean)
rmse(y_val, y_pred_mean)

In [ ]:
X_train = df_train.fillna(0).values
w0, w = train_linear_regression(X_train, y_train)
print(w0,w)

In [ ]:
def train_linear_regression_reg(X, y, r=0.001):
    ones = np.ones(X.shape[0])
    X = np.column_stack([ones, X])

    XTX = X.T.dot(X)
    XTX = XTX + r * np.eye(XTX.shape[0])

    XTX_inv = np.linalg.inv(XTX)
    w_full = XTX_inv.dot(X.T).dot(y)
    
    return w_full[0], w_full[1:]

for r in [0, 0.000001, 0.0001, 0.001, 0.01, 0.1, 1, 5, 10]:
    X_train = df_train.fillna(0).values
    w0, w = train_linear_regression_reg(X_train, y_train, r=r)

    X_val = df_val.fillna(0).values
    y_pred = w0 + X_val.dot(w)
    score = rmse(y_val, y_pred)
    
    print(r, w0, score)

In [ ]:
idx = np.arange(n)
np.random.seed(9)
np.random.shuffle(idx)

df_train = df_short.iloc[idx[:n_train+n_val]]
df_test = df_short.iloc[idx[n_train+n_val:]]

df_train = df_train.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

y_train = np.log1p(df_train.median_house_value.values)
y_test = np.log1p(df_test.median_house_value.values)

del df_train['median_house_value']
del df_test['median_house_value']
df_train.head()

X_train = df_train.fillna(0).values
w0, w = train_linear_regression_reg(X_train, y_train, r=0.001)

X_test = df_test.fillna(0).values
y_pred = w0 + X_test.dot(w)
score = rmse(y_test, y_pred)

print(score)

In [ ]:
score

In [ ]:
score